# Global Temperatures by Country ( Updated )
## by Junyoung Seo

Dataset from [Kaggle](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data/data)

This updated version automatically displays visualizations of global temperature data, cycling through each year's data every one second. This feature allows users to observe the historical trends of temperature changes across different countries and track the overall change in average temperature over time.

In [34]:
from dash import Dash, html, dcc, Input, Output, exceptions
import pandas as pd
import plotly.express as px

#reading csv file
df = pd.read_csv('GlobalLandTemperaturesByCountry.csv')
df['dt'] = pd.to_datetime(df['dt'])
df['year'] = df['dt'].dt.year

#selecting only after 1900
df = df[df['year'] >= 1900]

#creating a new dataframe with each countries average temperature by year.
df_avg = df.groupby(['Country', 'year'])['AverageTemperature'].mean().reset_index()

#creating the graphs
app = Dash('graph_with_timer')

#layout
app.layout = html.Div([
    dcc.Graph(id='average_temp_graph'),
    dcc.Slider(
        id='year_slider',
        min=1900,    #start with 1900
        max=df_avg['year'].max(),
        value=1900,
        marks={str(year): str(year) for year in range(1900, df_avg['year'].max()+1, 20)},   #marks by 20 years
        step=1
    ),
    dcc.Interval(
        id='timer_interval',
        interval=1000,  #update in every 1 second
    )
])

@app.callback(
    [Output('average_temp_graph', 'figure'),   #updating the graph
     Output('year_slider', 'value')],          #updating the slider too
    [Input('timer_interval', 'n_intervals')]  #automatically changing by every 1 second
)

def update_graph(n_intervals):
    if n_intervals is None:
        raise exceptions.PreventUpdate  #preventing the update if number of interval is none
        
        
    #calculating current year with number of intervals
    year = 1900 + n_intervals
    
    #creating new datafram with the values of average temperature of each countries in selected year.
    slider_df = df_avg[df_avg['year'] == year]
    
    #finding top 10 countries of average temperature
    top_10_countries = slider_df.nlargest(10, 'AverageTemperature')

    #creating the figure
    graph = px.bar(top_10_countries,
                   y='Country',
                   x='AverageTemperature',
                   color='AverageTemperature',   #chaing colors by the value of temperature
                   labels={'AverageTemperature': 'Average Temperature (°C)'},
                   title=f'Global Average Temperatures in {year}',
                   orientation='h',   #making the graph horizontal for the better visualization
                   color_continuous_scale="reds"   #setting red as a base color
                  )
    
    #fliping the graph upside down
    graph.update_yaxes(autorange="reversed"
                      )

    return graph, year  #return the current year and the graph

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)